Ok, so at the end of the last chapter, we found that the functional test's attempt to write to the table in our HTML to-do list was not saving the input.  Harry says that you need more than just Selenium to do that; you need to specify how to get the input into a database backing up the page.  You could use HTML5 or JavaScript, but we start with a simple HTML POST request.

The way this is done is actually to write it into our `home.html` template file, with an `input` possessing a `name` attribute, and wrapping that in a `<form method="POST">` tag.  Furthermore, this tag goes around your previously-entered `input` item with the `placeholder` attribute saying `"Enter a to-do item"`:

In [2]:
%%writefile lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Your To-Do list</h1>
        <form method="POST">
            <input id="id_new_item" placeholder="Enter a to-do item" />
        </form>
        <table id="id_list_table">
        </table>
    </body>
</html>


Overwriting lists/templates/home.html


In [4]:
# !python functional_tests.py

Yields

`selenium.common.exceptions.NoSuchElementException: Message: Unable to locate element: [id="id_list_table"]`

as anticipated by the text.

He suggests that, when confronted with an error that's hard to interpret, like this one, there are a few common steps to consider trying:

* Add `print` statements to get more info about the state (or, presumably, execute with debugging turned on in VS Code or other IDE)
* Try to write more context into your Error statements, to reflect the current state of the interpreter
* Try firing up your app server and just check the page yourself (i.e., not within a functional test)
* Use `time.sleep` to slow down the execution, giving you more time to visualize what's going on with the site

We actually already have the `time.sleep` option implemented in the functional test, so he suggests just extending it from 1 second as a quick sanity check.

In [6]:
%%writefile functional_tests.py
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import unittest

class NewVisitorTest(unittest.TestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get('http://localhost:8000')

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        time.sleep(60)

        table = self.browser.find_element_by_id('id_list_table')
        rows = table.find_elements_by_tag_name('tr')
        self.assertTrue(
            any(row.text == '1: Buy peacock feathers' for row in rows),
            "New to-do item did not appear in table"
        )

        # There is still a text box inviting her to add another item.
        # Shee enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        self.fail('Finish the test!')

        # The page updates again, and now shows both items on her list
        # ...

if __name__ == '__main__':
    unittest.main(warnings='ignore')


Overwriting functional_tests.py


In [8]:
!python functional_tests.py

E
ERROR: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 40, in test_can_start_a_list_and_retrieve_it_later
    table = self.browser.find_element_by_id('id_list_table')
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 360, in find_element_by_id
    return self.find_element(by=By.ID, value=id_)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 978, in find_element
    'value': value})['value']
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\errorhandler.py", li

Ok, doing that lets you see the page before the test checks and closes the browser.  Major text output is:

<blockquote><p>CSRF verification failed. Request aborted.</p><p>You are seeing this message because this site requires a CSRF cookie when submitting forms. This cookie is required for security reasons, to ensure that your browser is not being hijacked by third parties.</p><p>If you have configured your browser to disable cookies, please re-enable them, at least for this site, or for 'same-origin' requests.</p></blockquote>

That's a *bit* more context: something about cookies.  The text at this point has a box indicating that CSRF stands for "*Cross-Site Request Forgery*", a form of exploit.  There's no real way to anticipate what the heck it's about and how to deal with it without a deep dive into the web, but it turns out that there's a specific way that [Django handles CSRF](https://docs.djangoproject.com/en/4.0/ref/csrf/) that involves just inserting the template magic

`{% csrf_token %}`

into the POST request form, right after your `input` tag:

In [12]:
%%writefile lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Your To-Do list</h1>
        <form method="POST">
            <input id="id_new_item" placeholder="Enter a to-do item" />
            {% csrf_token %}
        </form>
        <table id="id_list_table">
        </table>
    </body>
</html>


Overwriting lists/templates/home.html


In [13]:
!python functional_tests.py

F
FAIL: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 44, in test_can_start_a_list_and_retrieve_it_later
    "New to-do item did not appear in table"
AssertionError: False is not true : New to-do item did not appear in table

----------------------------------------------------------------------
Ran 1 test in 66.777s

FAILED (failures=1)


So adding the `csrf_token` part was enough to get past the original error.  When I tried entering the expected text manually once the site launched, though, it still doesn't write it to any table.  Harry says that's because we haven't instructed Django on how to process any POST requests yet.  He says the way to do that is to write an `action=attribute` into the template; first, though, you add a unit test to `lists/tests.py` to anticipate this, and some conditional logic into the `home_page` func in the `lists/views.py` file.  So it's kind of a(nother) rabbit hole of HTML and web dev with which I'm unfamiliar and therefore complicates greatly the gist of understanding testing.  So instead of reading up on it and going off on a huge tangent about what all of this stuff means, I'll implement it verbatim from the text, and try to stick to the idea of understanding what it means in the context of TDD: how you'd anticipate the changes to the app code and write corresponding unit & functional tests.

First, we create a `test_can_save_a_POST_request` func for the unit tests file:

In [34]:
%%writefile lists/tests.py
from django.test import TestCase

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})
        self.assertIn('A new list item', response.content.decode())

Overwriting lists/tests.py


Then, the conditional logic for the `home_page` func in our `views` file:

In [32]:
%%writefile lists/views.py
from django.shortcuts import render
from django.http import HttpResponse

def home_page(request):
    if request.method == 'POST':
        return HttpResponse(request.POST['item_text'])
    return render(request, 'home.html')


Overwriting lists/views.py


In [35]:
# !python functional_tests.py
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


..
----------------------------------------------------------------------
Ran 2 tests in 0.026s

OK


All right, but then we turn around and say that's just to get the tests to pass, and not really the way you want to implement it after all.  ~~And, at present, it's not even passing in my hands.~~  (I had missed the detail that the change was only meant to get the **unit tests** to pass, not necessarily the functional tests... we're back in accord with the text.)  So move on and check out the more authoritative way forward, as revealed later in the chapter.

It involves leveraging more of the Django syntax for incorporating template text based on Python variables, which we've already seen with the whole `csrf_token` magic, earlier.  Except that if you're not using Django's magic, but instead native Python variables, apparently the syntax is to include in *double* curly braces:

In [63]:
%%writefile lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Your To-Do list</h1>
        <form method="POST">
            <input name="item_text" id="id_new_item" placeholder="Enter a to-do item" />
            {% csrf_token %}
        </form>
        <table id="id_list_table">
            <tr><td>{{ new_item_text }}</td></tr>
        </table>
    </body>
</html>


Overwriting lists/templates/home.html


With the explanation that `new_item_text` in the template is meant to distinguish it from the `item_text` specified in the `views.py` file in order to force us to realize that the one does not become the other automatically, but only via the action of interpreting the template via a view.

Then we modify the unit tests file again:

In [40]:
%%writefile lists/tests.py
from django.test import TestCase

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})
        self.assertIn('A new list item', response.content.decode())
        self.assertTemplateUsed(response, 'home.html')


Overwriting lists/tests.py


In [41]:
# !python functional_tests.py
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


F.
FAIL: test_can_save_a_POST_request (lists.tests.HomePageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 12, in test_can_save_a_POST_request
    self.assertTemplateUsed(response, 'home.html')
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\testcases.py", line 578, in assertTemplateUsed
    self.fail(msg_prefix + "No templates used to render the response")
AssertionError: No templates used to render the response

----------------------------------------------------------------------
Ran 2 tests in 0.024s

FAILED (failures=1)
E
ERROR: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 40, in test_can_start_a_list_and_retrieve_it_later
    table = self.browser.

Ok, unsure about the second error, but the text anticipates the first one, as the unit test no longer passing due to our original, overly-simplistic code written into the views file.  So we update that, and Harry says that the `django.shortcuts.render` func accepts an optional *third* arg, which should be a dict mapping the template names to specific values.

In [43]:
%%writefile lists/views.py
from django.shortcuts import render
from django.http import HttpResponse

def home_page(request):
    return render(request, 'home.html', {
        'new_item_text': request.POST['item_text']
    })


Overwriting lists/views.py


In [44]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


FE
ERROR: test_home_page_returns_correct_html (lists.tests.HomePageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\utils\datastructures.py", line 83, in __getitem__
    list_ = super(MultiValueDict, self).__getitem__(key)
KeyError: 'item_text'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 6, in test_home_page_returns_correct_html
    response = self.client.get('/')
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\client.py", line 536, in get
    **extra)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\client.py", line 340, in get
    return self.generic('GET', path, secure=secure, **r)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\s

# Unexpected Failure

Ok, so now we're confronted with an error that the author intended as an unexpected one (again, since I don't know a damn thing about Django/HTML, they're *all* unexpected to me); the code doesn't know what to do now when it *doesn't* receive a POST request.  His point is that if you don't write tests (or only write ones to catch the things you *expect* to be frequent failure cases), you may not anticipate the weird stuff that can happen due to your lack of imagination.

The fix is pretty simple; modify the third arg we just passed to `render` in the views file.  Specifically, instead of direct bracket-based indexing of our Python dict, use the `get` method, which in turn accepts a second arg as what to return should the key not be present in your dict (we want that default value to be an empty string, here):

In [55]:
%%writefile lists/views.py
from django.shortcuts import render
from django.http import HttpResponse

def home_page(request):
    return render(request, 'home.html', {
        'new_item_text': request.POST.get('item_text', '')
    })


Overwriting lists/views.py


In [56]:
!python functional_tests.py

F
FAIL: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 44, in test_can_start_a_list_and_retrieve_it_later
    "New to-do item did not appear in table"
AssertionError: False is not true : New to-do item did not appear in table

----------------------------------------------------------------------
Ran 1 test in 66.687s

FAILED (failures=1)


Also in accord with the text up to this point.  The author suggests further modifying the error statement encountered to be even more informative (by printing the content of the variable that it's checking):

In [61]:
%%writefile functional_tests.py
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import unittest

class NewVisitorTest(unittest.TestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get('http://localhost:8000')

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        time.sleep(30)

        table = self.browser.find_element_by_id('id_list_table')
        rows = table.find_elements_by_tag_name('tr')
        self.assertIn('1: Buy peacock feathers', [row.text for row in rows])
        # self.assertTrue(
        #     any(row.text == '1: Buy peacock feathers' for row in rows),
        #     f"New to-do item did not appear in table.  Contents were:\n{table.text}"
        # )

        # There is still a text box inviting her to add another item.
        # Shee enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        self.fail('Finish the test!')

        # The page updates again, and now shows both items on her list
        # ...

if __name__ == '__main__':
    unittest.main(warnings='ignore')


Overwriting functional_tests.py


In [64]:
!python functional_tests.py

F
FAIL: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 42, in test_can_start_a_list_and_retrieve_it_later
    self.assertIn('1: Buy peacock feathers', [row.text for row in rows])
AssertionError: '1: Buy peacock feathers' not found in ['1. Buy peacock feathers']

----------------------------------------------------------------------
Ran 1 test in 36.857s

FAILED (failures=1)


Ok, one issue that I encountered was that I hadn't copied the `input` tag over from the text correctly; it needed to be

            <input name="item_text" id="id_new_item" placeholder="Enter a to-do item" />

When I added the `<form method="POST">` earlier in the chapter, I had failed to add that `name="item_text"` attribute to the `input` tag within it.

Anyways, the author tried modifying the `test_can_start_a_list_and_retrieve_it_later` functional test at first by forcing his error statement to dump the contents of whatever the user had input, but then adds a note that someone else pointed out that an `assertIn` call there would be simpler than `assertTrue`.  And that using `assertIn` forces the error statement to include the variable you're inspecting whenever it fails, simplifying the test.  His overall point is that whenever you think you've found some clever solution for something, you're probably in fact just making the code overly complicated, and should search your docs/API/library for another method or entry point that's more directly relevant for the thing you're trying to do.

And then, the final lesson from this section is just that the test now fails for trivial reasons: the thing you're trying to ensure a user enters doesn't *exactly* match the string that the user may choose to input at a given time.  No general advice about avoiding this, though; seems dumb to hard-code a test that's quite so picky about formatting of the input string.  Maybe a regex would be more appropriate?

There's actually a bit more here; I was missing that we had already written in the automatic entry of `"Buy peacock feathers"` into the functional tests file, and was doing it manually when the browser opened (I had written in a much longer `time.sleep` period when troubleshooting that whole `csrf_token` thing, and apparently that length of delay in the browser closing had prompted my brain to enter the text manually whenever it opened).  Instead, he's referring to the mismatch between what the test says the user would enter and what it's searching for in the table.  So Harry's solution is just to hard-code in the prefix item number to the template file.  But that seems kind of dumb to me, if we're ultimately going to try to accommodate the entry of more items, where you'd want that item number to increment, so hard coding seems like something we'd just undo later, so I'm skipping that modification for now.

...Ah, well, maybe I'd better stick to the letter of the law here, and implement that change as indicated in the text, so as not to get divergent behavior where I subsequently forget about the discrepancy and have more headaches fixing it:

In [7]:
%%writefile lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Your To-Do list</h1>
        <form method="POST">
            <input name="item_text" id="id_new_item" placeholder="Enter a to-do item" />
            {% csrf_token %}
        </form>
        <table id="id_list_table">
            <tr><td>1: {{ new_item_text }}</td></tr>
        </table>
    </body>
</html>


Overwriting lists/templates/home.html


# More Inputs, and Refactor

At this point, Harry goes on to add more `assert` statements to the functional tests about appending to the To-Do list.  But the original approach tried gets kind of clunky, so we refactor to reduce repetition.  Specifically, he suggests pulling out the code about looking for specific text in the table to its own function, and having the func about storing and retrieving text call that for every line you expect to find in the table:

In [28]:
%%writefile functional_tests.py
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import unittest

class NewVisitorTest(unittest.TestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def check_for_row_in_list_table(self, row_text):
        table = self.browser.find_element_by_id('id_list_table')
        rows = table.find_elements_by_tag_name('tr')
        self.assertIn(row_text, [row.text for row in rows])

    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get('http://localhost:8000')

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        time.sleep(1)
        self.check_for_row_in_list_table('1: Buy peacock feathers')

        # There is still a text box inviting her to add another item.
        # She enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        inputbox = self.browser.find_element_by_id('id_new_item')
        inputbox.send_keys('Use peacock feathers to make a fly')
        inputbox.send_keys(Keys.ENTER)
        time.sleep(1)

        # The page updates again, and now shows both items on her list
        self.check_for_row_in_list_table('1: Buy peacock feathers')
        self.check_for_row_in_list_table('2: Use peacock feathers to make a fly')

        # Edith wonders whether the site will remember her list.  Then she sees
        # ...

if __name__ == '__main__':
    unittest.main(warnings='ignore')


Overwriting functional_tests.py


In [27]:
!python functional_tests.py

F
FAIL: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 55, in test_can_start_a_list_and_retrieve_it_later
    self.check_for_row_in_list_table('1: Buy peacock feathers')
  File "functional_tests.py", line 17, in check_for_row_in_list_table
    self.assertIn(row_text, [row.text for row in rows])
AssertionError: '1: Buy peacock feathers' not found in ['1: Use peacock feathers to make a fly']

----------------------------------------------------------------------
Ran 1 test in 13.358s

FAILED (failures=1)


Ok, I did hit an error referencing something "stale":

~~~bash
selenium.common.exceptions.StaleElementReferenceException: Message: The element reference of <tr> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed
~~~

On page 62 of the text, Harry says that if you see this you may need to increase the `sleep` period upstream of the error.  I'll try that now.  ...Ok, the key turned out to be not incrementing the original call on line 44 so much as adding a second call to sleep right after the *second* time that the `send_keys(Keys.ENTER)` was called.

So anyways, now we're seeing the error reported in the text, which (again) is a side-effect of the silly hard-coding done to get around the earlier error: we're calling everything the first element in the list when we add it.

# The Django ORM

We turn to calling out more functionality of Django that's intended to lighten the load (once you know how to use it) over writing straight HTML by hand.  In this case, that's the *Object-Relational Mapper (ORM)*.  Harry says that in this paradigm classes stand in for database tables, their attributes stand in for columns of your database tables, and each *instance* of such a class are to be thought of as the rows of your DB.

I guess that makes sense.  Plenty of elements that would fit neatly in the row of a relational DB would be like individual items you might track in an inventory, samples in LIMS, etc.  I hadn't heard it explained that way before.

To start to use it, of course, we begin by writing a unit test that assumes it will be used, and in this case it means a new class in our unit test file:

In [30]:
%%writefile lists/tests.py
from django.test import TestCase
from lists.models import Item

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})
        self.assertIn('A new list item', response.content.decode())
        self.assertTemplateUsed(response, 'home.html')

class ItemModelTest(TestCase):
    
    def test_saving_and_retrieving_items(self):
        first_item = Item()
        first_item.text = 'The first (ever) list item'
        first_item.save()

        second_item = Item()
        second_item.text = 'Item the second'
        second_item.save()

        saved_items = Item.objects.all()
        self.assertEqual(saved_items.count(), 2)

        first_saved_item = saved_items[0]
        second_saved_item = saved_items[1]
        self.assertEqual(first_saved_item.text, 'The first (ever) list item')
        self.assertEqual(second_saved_item.text, 'Item the second')

Overwriting lists/tests.py


Ok, so major lessons from that example:

1. The `lists.models` file was created by the call to `python manage.py startapp lists` way back in the third chapter
    * Right now, that file is empty, other than an the call to:
        * `from django.db import models`
    * ~~So that loads all of the classes & variables present in `django.db` under that namespace, which includes `Item` (even though there's no line referencing `Item` in `lists.models` itself)~~
    * ...Nope.  We've written it this way, but upon testing it will immediately raise an `ImportError`, so we'll subsequently modify the content of `lists.models` to subclass `models.Model` in order to make an empty `Item` class
2. So under the ORM, that `Item` instance should be a row of a DB, even though we haven't named that DB or anything.  And I don't know whether `text` is a default attr of `Item`s, or if `Item` permits arbitrary new declaration of attr names within lines extending it...
3. Anyways, to add rows to a DB with the ORM schema, you instantiate a new `Item`, store attrs representing the columns of the DB, and then call `save` to write it out.
4. Once a DB is populated, you can call `objects` on the class to get specific rows; the `all` method thereof returns all rows rather than some kind of strict subset.
    * Harry says that what's returned is a `QuerySet`, which behaves a lot like a `list`

He suggests you check out the Django [tutorial](https://docs.djangoproject.com/en/1.11/contents/) to get a peek at more features, but it's not easy for me to find the section relating to the ORM, specifically.

<div class="alert alert-info">There's a box here clarifying terminology, saying that "unit tests" strictly speaking only touch your source code, and not any resources not encoded in your codebase, including any databases (presumably because their state can depend on stuff not in your code itself?).  Any tests that rely on such external resources are technically called <i>integrated tests</i> rather than unit tests, but Harry will ignore the distinction for most of this book.</div>

## First Migration

According to the author, the job of building a database is handled under an abstraction called a *migration* (or "*migrations*"; maybe the plural is appropriate here).  This is actually invoked from the terminal, perhaps once per project, rather than as code you have to execute every time you start up the server?

~~~bash
python manage.py makemigrations
~~~

...Well, at least every time you add a new database "model" to the `lists/models.py` file; the output written to `stdout` when I ran that call indicates it was peeking inside that file.

~~~bash
(test_dev_book) C:\Data\projects\test_driven_development>python manage.py makemigrations
Migrations for 'lists':
  lists\migrations\0001_initial.py
    - Create model Item
~~~

So it creates a subdir in `lists` called `migrations`, and writes data to arbitrarily-named `.py` files within it.

It's implied that this should be enough to prepare us to re-test the unit tests again:

In [39]:
!python manage.py test lists

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


..E
ERROR: test_saving_and_retrieving_items (lists.tests.ItemModelTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 31, in test_saving_and_retrieving_items
    self.assertEqual(first_saved_item.text, 'The first (ever) list item')
AttributeError: 'Item' object has no attribute 'text'

----------------------------------------------------------------------
Ran 3 tests in 0.055s

FAILED (errors=1)


So the `Item` class *didn't* inherit all the attrs that our test assumed; we have to add them explicitly:

"*Classes that inherit from `models.Model` map to tables in the database.  By default they get an auto-generated `id` attribute, which will be a primary key column in the database, but you have to define any other columns you want explicitly`"...

In [41]:
%%writefile lists/models.py
from django.db import models

class Item(models.Model):
    text = models.TextField()


Overwriting lists/models.py


Harry says that common types that you may want to try when defining attrs/columns of your models/tables include

* `IntegerField`
* `CharField`
* `DateField`

and others.  The advantage motivating `TextField` over `CharField` here is that the former doesn't require an expected max length of characters it can contain.

If you test again after editing the `models` file, you'll see a failure because you need to re-run the `makemigrations` command again for your project.  If you try that as things stand *now*, however, you get a prompt in the terminal saying that it can't create that field until you tell what you want to use as a null value for any rows that end up missing it.  You can define it in the terminal, but the more stable response is to encode that in the `models.py` file, too.

In [42]:
%%writefile lists/models.py
from django.db import models

class Item(models.Model):
    text = models.TextField(default='')


Overwriting lists/models.py


Harry says that this will suffice to get the ORM to recognize the `text` attr on your objects as a special attribute, and let the unit tests pass:

In [43]:
!python manage.py test lists

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


...
----------------------------------------------------------------------
Ran 3 tests in 0.031s

OK


And we commit again.


# Saving POST to DB

Then we want to actually get the POST request to make a change in the DB.  So we adjust the unit test, adding a new func to our `ItemModelTest` class:

In [45]:
%%writefile lists/tests.py
from django.test import TestCase
from lists.models import Item

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})
        self.assertIn('A new list item', response.content.decode())
        self.assertTemplateUsed(response, 'home.html')

class ItemModelTest(TestCase):
    
    def test_saving_and_retrieving_items(self):
        first_item = Item()
        first_item.text = 'The first (ever) list item'
        first_item.save()

        second_item = Item()
        second_item.text = 'Item the second'
        second_item.save()

        saved_items = Item.objects.all()
        self.assertEqual(saved_items.count(), 2)

        first_saved_item = saved_items[0]
        second_saved_item = saved_items[1]
        self.assertEqual(first_saved_item.text, 'The first (ever) list item')
        self.assertEqual(second_saved_item.text, 'Item the second')

    def test_can_save_a_POST_request(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})

        self.assertEqual(Item.objects.count(), 1)
        new_item = Item.objects.first()
        self.assertEqual(new_item.text, 'A new list item')

        self.assertIn('A new list item', response.content.decode())
        self.assertTemplateUsed(response, 'home.html')

Overwriting lists/tests.py


He explains three of the new lines:

* 38 checks that you now have one row in your db
* 39 grabs the first row and assigns it to a new variable
* 40 checks that the content (text) of that row's `text` column is correct

He also notest that this whole test is getting a bit too long; contributing to a bad code smell that may justify refactoring at some point.

In [46]:
!python manage.py test

Creating test database for alias 'default'...

..F.
FAIL: test_can_save_a_POST_request (lists.tests.ItemModelTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 37, in test_can_save_a_POST_request
    self.assertEqual(Item.objects.count(), 1)
AssertionError: 0 != 1

----------------------------------------------------------------------
Ran 4 tests in 0.081s

FAILED (failures=1)



System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


So the result of line 38 is an expected failure, since we haven't written the app code that will pass the POST request along to the db.  Harry suggests that the problem at this point is that our `views` file needs adjustment:

In [48]:
%%writefile lists/views.py
from django.shortcuts import render
from lists.models import Item

def home_page(request):
    item = Item()
    item.text = request.POST.get('item_text', '')
    item.save()

    return render(request, 'home.html', {
        'new_item_text': request.POST.get('item_text', '')
    })


Overwriting lists/views.py


In [49]:
!python manage.py test

Creating test database for alias 'default'...

....
----------------------------------------------------------------------
Ran 4 tests in 0.067s

OK



System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


And now a successful passing of the unit test.  But he's noted multiple issues (including the bad code smell noted above):

1. Line 7 of the views file above means that we save an empty string for every request made
2. The overall unit test for the POST request is getting a bit long
3. It currently doesn't display "multiple items in the table"; unclear whether that means only one row or only one column
4. It would need more than one list

He has us start with addressing the first item, and checking whether any text needs saving.  It involves creating a new func within the `HomePageTest` of the unit test, and modifications to the `views` file, too:

In [2]:
%%writefile lists/tests.py
from django.test import TestCase
from lists.models import Item

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})
        self.assertIn('A new list item', response.content.decode())
        self.assertTemplateUsed(response, 'home.html')

    def test_only_saves_items_when_necessary(self):
        self.client.get('/')
        self.assertEqual(Item.objects.count(), 0)


class ItemModelTest(TestCase):
    
    def test_saving_and_retrieving_items(self):
        first_item = Item()
        first_item.text = 'The first (ever) list item'
        first_item.save()

        second_item = Item()
        second_item.text = 'Item the second'
        second_item.save()

        saved_items = Item.objects.all()
        self.assertEqual(saved_items.count(), 2)

        first_saved_item = saved_items[0]
        second_saved_item = saved_items[1]
        self.assertEqual(first_saved_item.text, 'The first (ever) list item')
        self.assertEqual(second_saved_item.text, 'Item the second')

    def test_can_save_a_POST_request(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})

        self.assertEqual(Item.objects.count(), 1)
        new_item = Item.objects.first()
        self.assertEqual(new_item.text, 'A new list item')

        self.assertIn('A new list item', response.content.decode())
        self.assertTemplateUsed(response, 'home.html')


Overwriting lists/tests.py


Which should yield an error.

In [3]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


..F..
FAIL: test_only_saves_items_when_necessary (lists.tests.HomePageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 17, in test_only_saves_items_when_necessary
    self.assertEqual(Item.objects.count(), 0)
AssertionError: 1 != 0

----------------------------------------------------------------------
Ran 5 tests in 0.048s

FAILED (failures=1)


Which we intend to address by modifying the views:

In [5]:
%%writefile lists/views.py
from django.shortcuts import render
from lists.models import Item

def home_page(request):
    if request.method == 'POST':
        new_item_text = request.POST['item_text']
        Item.objects.create(text=new_item_text)
    else:
        new_item_text = ''

    return render(request, 'home.html', {
        'new_item_text': new_item_text,
    })


Overwriting lists/views.py


Quick breakdown of the new lines:

1. #7 - whatever is getting passed in gets stored in `new_item_text` var
2. #8 - `objects.create` creates a new instance of `Item`, without having to call `save`

I guess the crux of it is that you're checking whether your `request` even has a `POST` method associated with it, and otherwise sets the var to an empty string, which should be the same as before, *except* that there's no `save` call, so it only writes to the DB if the request wasn't empty.


# Redirect After a POST

It's still weird syntax; why even bother setting an empty string if you're not going to save it?  Harry says that a view function is in charge of *both* processing user input *and* returning a relevant result.  He says the approach above was dedicated towards the former, saving whatever users input, and we'd like to tweak the latter purpose of returning a more streamlined result.  He introduces the [injunction to redirect after POST calls](https://en.wikipedia.org/wiki/Post/Redirect/Get).  So rather than immediately re-rendering the page with the user input saved in some kind of temporary state, you write to DB, and re-load the home page, drawing on the updated DB.

In [27]:
%%writefile lists/tests.py
from django.test import TestCase
from lists.models import Item

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})

        self.assertEqual(Item.objects.count(), 1)
        new_item = Item.objects.first()
        self.assertEqual(new_item.text, 'A new list item')

        self.assertEqual(response.status_code, 302)
        self.assertEqual(response['location'], '/')

    def test_only_saves_items_when_necessary(self):
        self.client.get('/')
        self.assertEqual(Item.objects.count(), 0)


class ItemModelTest(TestCase):
    
    def test_saving_and_retrieving_items(self):
        first_item = Item()
        first_item.text = 'The first (ever) list item'
        first_item.save()

        second_item = Item()
        second_item.text = 'Item the second'
        second_item.save()

        saved_items = Item.objects.all()
        self.assertEqual(saved_items.count(), 2)

        first_saved_item = saved_items[0]
        second_saved_item = saved_items[1]
        self.assertEqual(first_saved_item.text, 'The first (ever) list item')
        self.assertEqual(second_saved_item.text, 'Item the second')

Overwriting lists/tests.py


He says that, since we're now redirecting, we don't expect the `test_can_save_a_POST_request`'s call to `return render` based on the POST request directly, but rather to... well, it's getting too complicated for me.  But he says the updated test should currently return an error about the status code of the response:

In [16]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


F....
FAIL: test_can_save_a_POST_request (lists.tests.HomePageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 17, in test_can_save_a_POST_request
    self.assertEqual(response.status_code, 302)
AssertionError: 200 != 302

----------------------------------------------------------------------
Ran 5 tests in 0.028s

FAILED (failures=1)


And with that expected failure, we proceed to modify the view:

In [29]:
%%writefile lists/views.py
from django.shortcuts import redirect, render
from lists.models import Item

def home_page(request):
    if request.method == 'POST':
        Item.objects.create(text=request.POST['item_text'])
        return redirect('/')

    return render(request, 'home.html')

Overwriting lists/views.py


Multiple return statements.  Hmm.  Well, the behavior is clearer; if you see a POST request, pass it on, and then re-load the homepage.  Otherwise, just load the homepage.

In [30]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


....
----------------------------------------------------------------------
Ran 4 tests in 0.032s

OK


## Clean Up: Each Test Should only Test One Thing

We should reduce the number of assertions per test func in order to simplify bug tracking.  One reason is that the test stops as soon as it hits one failed assertion, without testing downstream assertions.  So we want to refactor the `test_can_save_a_POST_request` func that we just wrote, and break it out into more methods:

In [32]:
%%writefile lists/tests.py
from django.test import TestCase
from lists.models import Item

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        self.client.post('/', data={'item_text': 'A new list item'})

        self.assertEqual(Item.objects.count(), 1)
        new_item = Item.objects.first()
        self.assertEqual(new_item.text, 'A new list item')

    def test_redirects_after_POST(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})
        self.assertEqual(response.status_code, 302)
        self.assertEqual(response['location'], '/')

    def test_only_saves_items_when_necessary(self):
        self.client.get('/')
        self.assertEqual(Item.objects.count(), 0)


class ItemModelTest(TestCase):
    
    def test_saving_and_retrieving_items(self):
        first_item = Item()
        first_item.text = 'The first (ever) list item'
        first_item.save()

        second_item = Item()
        second_item.text = 'Item the second'
        second_item.save()

        saved_items = Item.objects.all()
        self.assertEqual(saved_items.count(), 2)

        first_saved_item = saved_items[0]
        second_saved_item = saved_items[1]
        self.assertEqual(first_saved_item.text, 'The first (ever) list item')
        self.assertEqual(second_saved_item.text, 'Item the second')


Overwriting lists/tests.py


In [33]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


.....
----------------------------------------------------------------------
Ran 5 tests in 0.029s

OK


Then, he says to address the third item in the to-do list from before; making sure that the template can display more than one list item:

In [35]:
%%writefile lists/tests.py
from django.test import TestCase
from lists.models import Item

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        self.client.post('/', data={'item_text': 'A new list item'})

        self.assertEqual(Item.objects.count(), 1)
        new_item = Item.objects.first()
        self.assertEqual(new_item.text, 'A new list item')

    def test_redirects_after_POST(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})
        self.assertEqual(response.status_code, 302)
        self.assertEqual(response['location'], '/')

    def test_only_saves_items_when_necessary(self):
        self.client.get('/')
        self.assertEqual(Item.objects.count(), 0)

    def test_displays_all_list_items(self):
        Item.objects.create(text='itemey 1')
        Item.objects.create(text='itemey 2')

        response = self.client.get('/')

        self.assertIn('itemey 1', response.content.decode())
        self.assertIn('itemey 2', response.content.decode())


class ItemModelTest(TestCase):
    
    def test_saving_and_retrieving_items(self):
        first_item = Item()
        first_item.text = 'The first (ever) list item'
        first_item.save()

        second_item = Item()
        second_item.text = 'Item the second'
        second_item.save()

        saved_items = Item.objects.all()
        self.assertEqual(saved_items.count(), 2)

        first_saved_item = saved_items[0]
        second_saved_item = saved_items[1]
        self.assertEqual(first_saved_item.text, 'The first (ever) list item')
        self.assertEqual(second_saved_item.text, 'Item the second')


Overwriting lists/tests.py


Which has an expected failure like "`itemey 1 not found in <html>...`":

In [36]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


.F....
FAIL: test_displays_all_list_items (lists.tests.HomePageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 32, in test_displays_all_list_items
    self.assertIn('itemey 1', response.content.decode())
AssertionError: 'itemey 1' not found in '<html>\n    <head>\n        <title>To-Do lists</title>\n    </head>\n    <body>\n        <h1>Your To-Do list</h1>\n        <form method="POST">\n            <input name="item_text" id="id_new_item" placeholder="Enter a to-do item" />\n            <input type=\'hidden\' name=\'csrfmiddlewaretoken\' value=\'5hDyfM5IHLtBvcjYS7rn7wWMs9nFsmlVY2w8qK0mjNhPEHwJrPiqf3aYXQAIh8yc\' />\n        </form>\n        <table id="id_list_table">\n            <tr><td>1: </td></tr>\n        </table>\n    </body>\n</html>\n'

----------------------------------------------------------------------
Ran 6 tests in 0.031s

FAILED (failures=

To address this, we actually modify the template file.  He says the Django/Jinja syntax for iterating through lists is still single curly brace and percent signs, but otherwise it's normal Python `for` syntax, *except that* in order to exit the loop you need a closing `endfor` statement:

In [39]:
%%writefile lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Your To-Do list</h1>
        <form method="POST">
            <input name="item_text" id="id_new_item" placeholder="Enter a to-do item" />
            {% csrf_token %}
        </form>
        <table id="id_list_table">
            {% for item in items %}
                <tr><td>1: {{ item.text }}</td></tr>
            {% endfor %}
        </table>
    </body>
</html>


Overwriting lists/templates/home.html


Then, we also need to modify how the view renders, passing the items input by the test to the template iteratively:

In [41]:
%%writefile lists/views.py
from django.shortcuts import redirect, render
from lists.models import Item

def home_page(request):
    if request.method == 'POST':
        Item.objects.create(text=request.POST['item_text'])
        return redirect('/')

    items = Item.objects.all()
    return render(request, 'home.html', {'items': items})


Overwriting lists/views.py


In [43]:
!python functional_tests.py

F
FAIL: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 25, in test_can_start_a_list_and_retrieve_it_later
    self.assertIn('To-Do', self.browser.title)
AssertionError: 'To-Do' not found in 'OperationalError at /'

----------------------------------------------------------------------
Ran 1 test in 6.601s

FAILED (failures=1)


But that error is actually anticipated in the text.  It brings up a point he wants to emphasize, that running your functional tests involves slightly different approaches than running the unit tests.  He suggests opening the url ("`http://localhost:8000`") manually in a window to see more context from Django, and it says "`no such table: lists_item`".

Basically, the issue turns out to be that, since modifying our template, we haven't *migrated* our database to get Django to update it.  This won't be detected when running unit tests because Django sets up its own, separate test databases when running unit tests, but when you're running the functional test, you should be checking the actual database that your webpage/app is using.

So actually, and maybe a little surprisingly, this project actually hasn't created a valid database at this point.  Harry says that "*SQLite databases are just a file on disk, and you'll see in `settings.py` that Django, by default, will just put it in a file called `db.sqlite3` in the base project directory*".

There's text in the book here taht indicates a dict called `DATABASES`; it's unclear if it was supposed to have been created automatically by some specific command, or if we're supposed to add it ourselves.  After loading the `settings` file below, I don't see it, so I'll add it manually (BTW, there's a docstring pointing to the docs [here](https://docs.djangoproject.com/en/1.11/ref/settings/#databases) with the relevant syntax for what the dict may contain):

In [45]:
%%writefile superlists/settings.py
"""
Django settings for superlists project.

Generated by 'django-admin startproject' using Django 1.11.25.

For more information on this file, see
https://docs.djangoproject.com/en/1.11/topics/settings/

For the full list of settings and their values, see
https://docs.djangoproject.com/en/1.11/ref/settings/
"""

import os

# Build paths inside the project like this: os.path.join(BASE_DIR, ...)
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))


# Quick-start development settings - unsuitable for production
# See https://docs.djangoproject.com/en/1.11/howto/deployment/checklist/

# SECURITY WARNING: keep the secret key used in production secret!
SECRET_KEY = '&3s70fkdcir5wp&^vk4_hp^4-xwvovk&ms2!kzw+$)^o*8k+-('

# SECURITY WARNING: don't run with debug turned on in production!
DEBUG = True

ALLOWED_HOSTS = []


# Application definition

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'lists',
]

MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]

ROOT_URLCONF = 'superlists.urls'

TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]

WSGI_APPLICATION = 'superlists.wsgi.application'


# Database
# https://docs.djangoproject.com/en/1.11/ref/settings/#databases

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}


# Password validation
# https://docs.djangoproject.com/en/1.11/ref/settings/#auth-password-validators

AUTH_PASSWORD_VALIDATORS = [
    {
        'NAME': 'django.contrib.auth.password_validation.UserAttributeSimilarityValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.MinimumLengthValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.CommonPasswordValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.NumericPasswordValidator',
    },
]


# Internationalization
# https://docs.djangoproject.com/en/1.11/topics/i18n/

LANGUAGE_CODE = 'en-us'

TIME_ZONE = 'UTC'

USE_I18N = True

USE_L10N = True

USE_TZ = True


# Static files (CSS, JavaScript, Images)
# https://docs.djangoproject.com/en/1.11/howto/static-files/

STATIC_URL = '/static/'

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}

Overwriting superlists/settings.py


Then, to apply these changes, we need to call `manage.py migrate`:

In [46]:
!python manage.py migrate

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, lists, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying lists.0001_initial... OK
  Applying lists.0002_item_text... OK
  Applying sessions.0001_initial... OK


That gets rid of the error seen when opening the browser yourself; also re-try the functional test:

In [47]:
!python functional_tests.py

F
FAIL: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 56, in test_can_start_a_list_and_retrieve_it_later
    self.check_for_row_in_list_table('2: Use peacock feathers to make a fly')
  File "functional_tests.py", line 17, in check_for_row_in_list_table
    self.assertIn(row_text, [row.text for row in rows])
AssertionError: '2: Use peacock feathers to make a fly' not found in ['1: Buy peacock feathers', '1: Use peacock feathers to make a fly']

----------------------------------------------------------------------
Ran 1 test in 12.763s

FAILED (failures=1)


Obviously, another contrived example to offer a teaching point.  In this case, it's that the numbering as rendered doesn't match what's expected by the test, and the preferred way to offload such manual numbering is to modify the Jinja syntax in the template file that's executing the `for` loop, to use a `forloop.counter` var to store which number we're on:

In [49]:
%%writefile lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Your To-Do list</h1>
        <form method="POST">
            <input name="item_text" id="id_new_item" placeholder="Enter a to-do item" />
            {% csrf_token %}
        </form>
        <table id="id_list_table">
            {% for item in items %}
                <tr><td> {{ forloop.counter }}: {{ item.text }}</td></tr>
            {% endfor %}
        </table>
    </body>
</html>


Overwriting lists/templates/home.html


In [50]:
!python functional_tests.py

.
----------------------------------------------------------------------
Ran 1 test in 11.863s

OK


But the text points out at this juncture that we're seeing duplicate messages of the numbered items in the browser window that pops up, even though the logic in the functional tests file passes.  The point is that the database itself is logging new entries every time we run it, and they'll continue to pile up.  We would prefer having the database wiped (at least of any entries made by the functional tests) whenever such a test concludes.  For now, he instead has us delete the `db.sqlite3` file, execute another `migrate` command, with the additional flag `--noinput`, with no explanation why that's required.  But apparently the database files will be successfully regenerated by the `migrate` call, now that we've modified the `settings` file.

In [51]:
import os

# os.remove('db.sqlite3')

In [52]:
!python manage.py migrate --noinput

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, lists, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying lists.0001_initial... OK
  Applying lists.0002_item_text... OK
  Applying sessions.0001_initial... OK


And finally, he suggests another commit to round out Chapter 5.